In [1]:
import urllib.request
import requests

query = '세탁기'
keys = []

with open('API_KEY.txt', 'r') as key:
    for line in key.readlines():
        text = line.strip()
        if text != '':
            keys.append(text.split(':')[1])

headers = {
    'X-Naver-Client-Id' : keys[0],
    'X-Naver-Client-Secret' : keys[1],
}
            
url = f'https://openapi.naver.com/v1/search/shop?&query={query}'
text = requests.get(url, headers=headers)
line = text.json()

In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

for i in line['items']:
    data = requests.get(i['link'])
    print(i['link'])
#     soup = BeautifulSoup(data.text)
#     x_path = ''
    
#     soup.find_element_by_xpath()
#     div = soup.select_one('div.list_basis')
#     a = div.select('a.basicList_etc__LSkN_')
    
    
    

https://search.shopping.naver.com/gate.nhn?id=21225654919
https://search.shopping.naver.com/gate.nhn?id=16567044284
https://search.shopping.naver.com/gate.nhn?id=16027057006
https://search.shopping.naver.com/gate.nhn?id=26854021527
https://search.shopping.naver.com/gate.nhn?id=21126513760
https://search.shopping.naver.com/gate.nhn?id=23034941490
https://search.shopping.naver.com/gate.nhn?id=23057107491
https://search.shopping.naver.com/gate.nhn?id=25661530523
https://search.shopping.naver.com/gate.nhn?id=23760142490
https://search.shopping.naver.com/gate.nhn?id=24567866524


In [ ]:
# 네이버 검색 사이트
import webbrowser
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

# naver_search = 'https://search.naver.com/search.naver?query='
# search_word = '고양이'



params = {
    'query' : '세탁기',
    'sort' : 'timestamp',
}


url = 'https://search.shopping.naver.com/search/all'

data = requests.get(url, params=params)

soup = BeautifulSoup(data.text)
div = soup.select_one('div.list_basis')
a = div.select('a.basicList_etc__LSkN_')

for i in a:
    print(i)
        

In [ ]:
import webbrowser

url = 'https://shopping.naver.com/home'
webbrowser.open(url)

In [ ]:
# 단일 크롤링
import time


driver = webdriver.Chrome()
driver.get("https://www.google.co.kr/imghp?hl=ko&ogbl") # 이미지 사이트를 한 번에 접속
# assert "Python" in driver.title

elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys("해리포터") # 검색어를 확인
elem.send_keys(Keys.RETURN) # enter 키를 친다


driver.find_elements(By.CSS_SELECTOR, '.rg_i.Q4LuWd')[0].click() # 큰 그림을 클릭
time.sleep(3)
src = driver.find_element(By.CSS_SELECTOR, '.n3VNCb').get_attribute('src') # src 주소를 가져와야한다.

# 이미지 다운로드
u.request.urlretrieve(src, 'test.png')



# n3VNCb KAlRDb



# https://upload.wikimedia.org/wikipedia/ko/d/dd/%ED%95%B4%EB%A6%AC_%ED%8F%AC%ED%84%B0%EC%99%80_%EB%A7%88%EB%B2%95%EC%82%AC%EC%9D%98_%EB%8F%8C_%EC%98%81%ED%99%94.jpg

# assert "No results found." not in driver.page_source
# driver.close()